In [20]:
from mp_api.client import MPRester
from pymatgen.core import Structure, Lattice, Element, Composition
from pymatgen.io.vasp.sets import MPRelaxSet
from pymatgen.analysis.structure_matcher import StructureMatcher as SM
from pymatgen.analysis.structure_matcher import FrameworkComparator

import os
import re
import sys
import json
import shutil
import numpy as np
from time import time
from glob import glob
from pprint import pprint
from copy import deepcopy
from monty.serialization import dumpfn,loadfn

In [27]:
def read_json(fjson):
    with open(fjson) as f:
        return json.load(f)


def write_json(d, fjson):
    with open(fjson, 'w') as f:
        json.dump(d, f)
    return


def get_inputs():

    for mpid in mpid_directory:

        struc = Structure.from_file(os.path.join(dope, 'POSCAR'))
        relax_set = MPRelaxSet(structure=struc)
        relax_set.write_input(output_dir=dope, potcar_spec=False)
        job_path = os.path.join(dope, 'job.sh')
        shutil.copyfile(sample_job, job_path)

    return

In [8]:
AnionLst=['P','S','Si','B','O'];
NotInclude=['H','Li','Be','C','N','F','Cl','Br','I','At','Ts','K','Rb','Cs','Fr','Ra',\
            'Se','Te','Po','Lv','As','Bi','Mc','Pb','Fl','Nh','Tl',\
            'Tc','Ru','Rh','Pd','Ag','Cd','Re','Os','Pt','Au','Hg',\
            'Rf','Db','Sg','Bh','Hs','Mt','Ds','Rg','Cn',\
            'Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu',\
            'Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf','Es','Fm','Md','No','Lr'];
NotInclude = list(set(NotInclude) - set(['Ag', 'Au', 'Hg', 'Os', 'Pd', 'Pt', 'Re', 'Rh', 'Ru', 'Tc']))
TmInclude = ['Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu',
             'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag',
             'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg']

In [10]:
#TODO: Need to check ICSD tag (or experimentally observed) from materials project data.

EntryDict={};
for NEle in range(4,8):
    with MPRester('dPcAQJZ6y1NZidGuvTerIPPFXHtsOb3E') as mpr: 
        EntryDict[NEle]=mpr.get_entries(chemsys_formula_mpids="Li-O-*",
                                        additional_criteria={"num_elements":NEle},
                                        inc_structure=None);

Retrieving ThermoDoc documents:   0%|          | 0/23407 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/5705 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/1000 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/35 [00:00<?, ?it/s]

In [43]:
# write_json(EntryDict, '/Users/yun/Berkeley/Codes/berkeley-ai-hack/data/raw_data.json')

In [15]:
###1-2. Exclude compounds that contain elements in NotInclude
NTotal=0;
StrLst=[]
for NEle, Entries in EntryDict.items():
    print('{} Entries found for NEle={}'.format(len(Entries), NEle));
    ValidMaterNum=0
    for EntryInd, Entry in enumerate(Entries):
        Comp=Entry.composition
        EleLst=list(Comp.get_el_amt_dict().keys())
        IsValid=True
        for Ele in EleLst:
            if Ele in TmInclude:
                IsValid=False
                break
        if not IsValid: 
            continue
        StrLst.append(Entry.structure); # Structure method under computedstructureentry
        ValidMaterNum+=1
    print('{} materials found for NEle={}'.format(ValidMaterNum,NEle))
    NTotal+=ValidMaterNum
print('{} Li Oxide Compounds found'.format(NTotal))

23408 Entries found for NEle=4
2330 materials found for NEle=4
5705 Entries found for NEle=5
665 materials found for NEle=5
1000 Entries found for NEle=6
94 materials found for NEle=6
35 Entries found for NEle=7
10 materials found for NEle=7
3099 Li Oxide Compounds found


In [26]:
tojson = []

for structure in StrLst:
    tojson.append(structure.as_dict())

In [46]:
write_json(tojson, '/Users/yun/Berkeley/Codes/berkeley-ai-hack/data/raw_data_Li.json')

In [49]:
StrLst[0].composition.to_pretty_string()

'Li1Al1As1O4'

In [ ]:
main_dir = '/Users/yun/Berkeley/Codes/berkeley-ai-hack/data'
save_dir = os.path.join(main_dir, 'li_electrolyte')

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for structure in StrLst:
    dir_name = structure.composition.to_pretty_string()
    spec_dir = os.path.join(save_dir, dir_name)
    
    struc = structure
    relax_set = MPRelaxSet(structure=struc)
    relax_set.write_input(output_dir=spec_dir, potcar_spec=True)

In [ ]:
import re
import sys
import json
import shutil

from glob import glob
from pymatgen.core import Structure, Lattice, Element, Composition
from pymatgen.io.vasp.sets import MPRelaxSet


def get_inputs():

    sample_job = '/home/yychoi/mv_conductor/doping/ca/sample.sh'
    mpid_directory = glob("./*/")
    for mpid in mpid_directory:
        dope_directory = glob(mpid + '*/')
        # print(dope_directory)
        for dope in dope_directory:
            struc = Structure.from_file(os.path.join(dope, 'POSCAR'))
            relax_set = MPRelaxSet(structure=struc)
            relax_set.write_input(output_dir=dope, potcar_spec=False)
            job_path = os.path.join(dope, 'job.sh')
            shutil.copyfile(sample_job, job_path)

    return


def specific():

    directory = "/home/yychoi/mv_conductor/doping/ca/mp-1218392/original"
    poscar = os.path.join(directory, "POSCAR")
    struc = Structure.from_file(poscar)
    relax_set = MPRelaxSet(structure=struc)
    relax_set.write_input(output_dir=directory, potcar_spec=False)

    return


def get_sub():

    sample_job = '/home/yychoi/mv_conductor/doping/ca'
    job_path = os.path.join(path_name, 'job.sh')
    shutil.copyfile(sample_job, job_path)

    return